In [1]:
import s3fs
from os.path import dirname, join
import time
import xarray as xr
import requests
import datetime
import numpy as np

# Subroutines

In [2]:
def make_encodings(G, netcdf_fill_value=-999, extra_prints=True):
    # G is the xarray dataset
    # PROVIDE SPECIFIC ENCODING DIRECTIVES FOR EACH DATA VAR
    
    dv_encoding = {}
    for dv in G.data_vars:
        dv_encoding[dv] = {'zlib':True,
                           'complevel':5,
                           'shuffle':True,
                           'dtype':'float32',
                           '_FillValue':netcdf_fill_value}


        # overwrite default coordinates attribute (PODAAC REQUEST)
        #G[dv].encoding['coordinates'] = dv_coordinate_attrs[dv]


    # PROVIDE SPECIFIC ENCODING DIRECTIVES FOR EACH COORDINATE
    if extra_prints: print('\n... creating coordinate encodings')
    coord_encoding = {}


    for coord in G.coords:
        # default encoding: no fill value, float32
        coord_encoding[coord] = {'_FillValue':None, 'dtype':'float32'}


        if (G[coord].values.dtype == np.int32) or (G[coord].values.dtype == np.int64):
            coord_encoding[coord]['dtype'] ='int32'


        if coord == 'time' or coord == 'time_bnds':
            coord_encoding[coord]['dtype'] ='int32'


            if 'units' in G[coord].attrs:
                # apply units as encoding for time
                coord_encoding[coord]['units'] = G[coord].attrs['units']
                # delete from the attributes list
                del G[coord].attrs['units']


        elif coord == 'time_step':
            coord_encoding[coord]['dtype'] ='int32'


    # MERGE ENCODINGS for coordinates and variables
    encoding = {**dv_encoding, **coord_encoding}

    return encoding

# Dask

In [3]:
from dask.distributed import Client, LocalCluster
client = Client("tcp://100.104.70.36:8786")
client

<Client: 'tcp://100.104.70.36:8786' processes=16 threads=16, memory=61.97 GiB>

In [3]:
!cat ~/.netrc | grep 'urs.earthdata.nasa.gov' | wc -l

1


In [4]:
def store_aws_keys(endpoint: str="https://archive.podaac.earthdata.nasa.gov/s3credentials"):    
    with requests.get(endpoint, "w") as r:
        accessKeyId, secretAccessKey, sessionToken, expiration = list(r.json().values())

    creds ={}
    creds['AccessKeyId'] = accessKeyId
    creds['SecretAccessKey'] = secretAccessKey
    creds['SessionToken'] = sessionToken
    creds['expiration'] = expiration
    
    return creds

creds = store_aws_keys()

ValueError: not enough values to unpack (expected 4, got 1)

In [ ]:
print(f"\nThe current session token expires at {creds['expiration']}.\n")
print(datetime.datetime.now())

In [5]:
ShortName = "OSCAR_L4_OC_FINAL_V2.0"

response = requests.get(
    url='https://cmr.earthdata.nasa.gov/search/collections.umm_json', 
    params={'provider': "POCLOUD",
            'ShortName': ShortName,
            'page_size': 1}
)
s3 = s3fs.S3FileSystem(
    key=creds['AccessKeyId'],
    secret=creds['SecretAccessKey'],
    token=creds['SessionToken'],
    client_kwargs={'region_name':'us-west-2'},
)
fs = s3fs.S3FileSystem(anon=False,
                      key=creds['AccessKeyId'],
                      secret=creds['SecretAccessKey'],
                      token=creds['SessionToken'])

NameError: name 'creds' is not defined

In [8]:
year=1993
month=1
day=1
oscar_files = fs.glob(join("podaac-ops-cumulus-protected/", ShortName, '*'+str(year)+str(month).zfill(2)+str(day).zfill(2)+'*.nc'))
paths=[fs.open(f) for f in oscar_files]
paths

[<File-like object S3FileSystem, podaac-ops-cumulus-protected/OSCAR_L4_OC_FINAL_V2.0/oscar_currents_final_19930101.nc>]

In [9]:
start_time = time.time()
oscar_final = xr.open_mfdataset(paths,
                                parallel=True, 
                                data_vars='minimal', 
                                coords='minimal',
                                compat='override')

print(time.time() - start_time)
print((time.time() - start_time)/len(paths))

2.1245386600494385
2.1246860027313232


In [10]:
encodings = make_encodings(oscar_final)


... creating coordinate encodings


In [11]:
encodings

{'u': {'zlib': True,
  'complevel': 5,
  'shuffle': True,
  'dtype': 'float32',
  '_FillValue': -999},
 'v': {'zlib': True,
  'complevel': 5,
  'shuffle': True,
  'dtype': 'float32',
  '_FillValue': -999},
 'ug': {'zlib': True,
  'complevel': 5,
  'shuffle': True,
  'dtype': 'float32',
  '_FillValue': -999},
 'vg': {'zlib': True,
  'complevel': 5,
  'shuffle': True,
  'dtype': 'float32',
  '_FillValue': -999},
 'lat': {'_FillValue': None, 'dtype': 'float32'},
 'lon': {'_FillValue': None, 'dtype': 'float32'},
 'time': {'_FillValue': None, 'dtype': 'int32'}}

In [12]:
oscar_final = oscar_final.load()

In [13]:
oscar_final

<xarray.Dataset> Size: 33MB
Dimensions:  (latitude: 719, longitude: 1440, time: 1)
Coordinates:
    lat      (latitude) float64 6kB -89.75 -89.5 -89.25 ... 89.25 89.5 89.75
    lon      (longitude) float64 12kB 0.0 0.25 0.5 0.75 ... 359.2 359.5 359.8
  * time     (time) object 8B 1993-01-01 00:00:00
Dimensions without coordinates: latitude, longitude
Data variables:
    u        (time, longitude, latitude) float64 8MB nan nan nan ... nan nan nan
    v        (time, longitude, latitude) float64 8MB nan nan nan ... nan nan nan
    ug       (time, longitude, latitude) float64 8MB nan nan nan ... nan nan nan
    vg       (time, longitude, latitude) float64 8MB nan nan nan ... nan nan nan
Attributes: (12/36)
    title:                      Ocean Surface Current Analyses Real-time (OSC...
    summary:                    Global, daily, 0.25 degree geostrophic and to...
    keywords:                   ocean currents,ocean circulation,surface curr...
    Conventions:                CF-1.8 Standard Names v77, ACDD-1.3, netcdf 4...
    id:                         OSCAR_L4_OC_FINAL_V2.0
    history:                    OSCAR 0.25 degree daily version 2.0 replaces ...
    ...                         ...
    geospatial_lon_max:         359.75
    geospatial_lon_units:       degrees_east
    geospatial_lon_resolution:  0.25 degree
    time_coverage_start:        1993-01-01T00:00:00
    time_coverage_end:          1993-01-01T23:59:59
    date_created:               2022-01-07

In [14]:
oscar_final.to_netcdf('/home/jpluser/efs-mount-point/sevfour/oscar/encodedfiles/test.nc', 
                      encoding=encodings)